In [1]:
import sys,os
import glob
import torch,dill
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from iresnet import iResBlock
from spectral_norm import SN

In [2]:
paths = glob.glob(os.path.expanduser('~/tb-experiments/iresnet_inv_test/**/*.trainer'),recursive=True)
print(paths)

['/home/marc/tb-experiments/iresnet_inv_test/CIFAR10/iResnet/s0.5/trial0/checkpoints/c10.trainer']


In [3]:
trainer = torch.load(paths[-1],pickle_module=dill)
trainer.model.eval();

/home/marc/anaconda3/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'iresnet.iResnet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/marc/anaconda3/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'auto_inverse.SequentialWithAutoInverseAndLogDet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/marc/anaconda3/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'iresnet.both' has changed. you can retrieve the original source code by accessing the object's source attribute 

In [ ]:
def change_tol(module):
    if isinstance(module,iResBlock):
        module.inverse_tol=1e-5
    if isinstance(module,SN):
        print(module._s)

In [ ]:
for mb in trainer.dataloaders['dev']:
    break
inp =x=mb[0]
trainer.model.apply(change_tol);

In [ ]:
with torch.no_grad():
    output = trainer.model[1].body(inp)

In [ ]:
sampled_imgs = trainer.model[1].sample(32).cpu().data

In [ ]:
grid = vutils.make_grid(sampled_imgs, normalize=True)
plt.imshow(grid.permute(1,2,0))

In [ ]:
grid.shape

In [ ]:
list(z_.shape for z_ in output)

In [ ]:
with torch.no_grad():
    inp_reconstructed = trainer.model[1].body.inverse(output);

In [ ]:
((inp_reconstructed[0] - inp[0]).norm()/(inp[0]).norm())

In [ ]:
plt.hist(inp_reconstructed[0].cpu().data.numpy().reshape(-1),alpha=.5)
plt.hist(inp[0].cpu().data.numpy().reshape(-1),alpha=.5)

In [ ]:
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
#print(y.shape)
grid = vutils.make_grid(mb[0]).cpu().data.numpy().transpose(1,2,0)
plt.imshow(grid)
plt.show()
grid = vutils.make_grid(inp_reconstructed).cpu().data.numpy().transpose(1,2,0)
plt.imshow(grid)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
for zi in z: print(zi.shape)